# Imports and Configs

In [83]:
import mysql.connector
import os
import pandas as pd
import yfinance as yf
import yaml
from dotenv import load_dotenv

load_dotenv()

True

In [96]:
# Import config file

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    start_date = config['start_date']
    securities = config['securities']
    transactions = config['transactions']
    currencies = config['currencies']

end_date = pd.to_datetime('today').date()

In [85]:
# Set up SQL database connection

connection = mysql.connector.connect(
    host=os.getenv('DB_HOSTNAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    port=os.getenv('DB_PORT'),
    database="Fund"
)

cursor = connection.cursor()

# Securities

In [86]:
# Create Securities SQL Table

cursor.execute("""
CREATE TABLE IF NOT EXISTS Securities (
    ticker VARCHAR(10) PRIMARY KEY,
    name VARCHAR(100),
    type VARCHAR(20) NOT NULL,
    geography VARCHAR(50),
    sector VARCHAR(50),
    currency CHAR(3) NOT NULL
);
""")
connection.commit()

In [87]:
# Backfill Securities Table

for ticker in securities:
    data = yf.Ticker(ticker)
    name = data.info['longName']
    # sector = data.info['sector'] # Often missing
    geography = data.info['region'] # Usually US even if Canadian (probably should specify in config)
    type = data.info['typeDisp']
    currency = data.info['currency']

    cursor.execute("""
    INSERT INTO Securities (ticker, name, type, geography, sector, currency)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    ticker = VALUES(ticker),
    name = VALUES(name),
    type = VALUES(type),
    geography = VALUES(geography),
    sector = VALUES(sector),
    currency = VALUES(currency);
    """, (ticker, name, type, geography, None, currency))
    connection.commit()

In [82]:
# Drop Securities SQL Table

cursor.execute("DROP TABLE Securities")
connection.commit()

# Transactions

In [88]:
# Create Transactions SQL Table

cursor.execute("""
CREATE TABLE IF NOT EXISTS Transactions (
    transaction_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    ticker VARCHAR(10) NOT NULL,
    date DATE NOT NULL,
    action ENUM('BUY', 'SELL') NOT NULL,
    shares INTEGER NOT NULL CHECK (shares > 0),
    price DECIMAL(20,10) NOT NULL CHECK (price > 0),
    currency CHAR(3) NOT NULL,
    FOREIGN KEY (ticker) REFERENCES Securities(ticker),
    UNIQUE (date, ticker, action)
);
""")
connection.commit()

In [99]:
# Backfill Transactions Table

for transaction in transactions:
    ticker = transaction['ticker']
    date = transaction['date']
    action = transaction['type']
    shares = transaction['quantity']
    price = transaction['price']
    currency = transaction['currency']

    cursor.execute("""
    INSERT INTO Transactions (ticker, date, action, shares, price, currency)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
    shares = VALUES(shares), 
    price = VALUES(price), 
    currency = VALUES(currency);
    """, (ticker, date, action, shares, price, currency))
    connection.commit()

In [81]:
# Drop Transactions SQL Table

cursor.execute("DROP TABLE Transactions")
connection.commit()

# Currencies

In [108]:
# Create Currencies SQL Table

cursor.execute("""
CREATE TABLE IF NOT EXISTS Currencies (
    date DATE NOT NULL PRIMARY KEY,
    USD DECIMAL(20,10) NOT NULL CHECK (USD > 0)
);
""")
connection.commit()

In [ ]:
# Backfill Currencies Table

for currency in currencies:
    ticker = currency['ticker']
    data = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
    data.index = data.index.date

    cursor.executemany("""
    INSERT INTO Currencies (date, USD)
    VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE
    USD = VALUES(USD);
    """, [(date, rate) for date, rate in data.items()])
    connection.commit()

In [107]:
# Drop Currencies SQL Table

cursor.execute("DROP TABLE Currencies")
connection.commit()

# Dates

In [104]:
# Create Dates SQL Table

cursor.execute("""
CREATE TABLE IF NOT EXISTS Dates (
    date DATE PRIMARY KEY
);
""")
connection.commit()

In [ ]:
# Backfill Dates Table

sp500 = yf.Ticker('^GSPC').history(start=start_date, end=end_date)
tsx = yf.Ticker('^GSPTSE').history(start=start_date, end=end_date)

sp500.index = pd.to_datetime(sp500.index).tz_localize(None)
tsx.index = pd.to_datetime(tsx.index).tz_localize(None)

valid_dates = sp500.index.union(tsx.index)
valid_dates = [date.date() for date in valid_dates]

cursor.executemany("""
    INSERT INTO Dates (date)
    VALUES (%s)
    ON DUPLICATE KEY UPDATE date = VALUES(date);
""", [(date,) for date in valid_dates])
connection.commit()

In [103]:
# Drop Dates SQL Table

cursor.execute("DROP TABLE Dates")
connection.commit()